# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os

import numpy as np
import pandas as pd

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset

In [2]:
pd.__version__

'1.1.5'

In [3]:
#files and directories
os.listdir(os.curdir)


['.amlignore',
 '.amlignore.amltmp',
 '.ipynb_aml_checkpoints',
 'hyperparameter_tuning (1).ipynb',
 'hyperparameter_tuning (1).ipynb.amltmp',
 'keras_env.yml',
 'keras_env.yml.amltmp',
 'keras_train.py',
 'keras_train.py.amltmp',
 'myenv.yml',
 'myenv.yml.amltmp',
 'scoring.py',
 'test.pkl',
 'train.pkl']

# Initialize the Workspace and an Experiment


In [4]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

experiment_name = 'keras_housing'
project_folder = '.'
os.makedirs(project_folder, exist_ok=True)

experiment = Experiment(ws, experiment_name)
experiment

quick-starts-ws-247912
aml-quickstarts-247912
southcentralus
48a74bb7-9950-4cc1-9caa-5d50f995cc55


Name,Workspace,Report Page,Docs Page
keras_housing,quick-starts-ws-247912,Link to Azure Machine Learning studio,Link to Documentation


# Create or Attach a Compute Resource

In [5]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "notebook247912"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)
# For a more detailed view of current AmlCompute status, use get_status().

Found existing cluster, use it.

Running


# Environment Set Up

In [6]:
from azureml.core import Environment

keras_env = Environment.from_conda_specification(name = 'keras-2.3.1', file_path = 'keras_env.yml')

# Specify a GPU base image
# keras_env.docker.enabled = True
# keras_env.docker.base_image = 'mcr.microsoft.com/azureml/openmpi3.1.2-cuda10.0-cudnn7-ubuntu18.04'

In [7]:
from azureml.train.hyperdrive import RandomParameterSampling, BanditPolicy, HyperDriveConfig, PrimaryMetricGoal
from azureml.core import ScriptRunConfig
from azureml.train.hyperdrive import choice, loguniform
from azureml.widgets import RunDetails

## Dataset

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import pandas as pd
data = Dataset.get_by_name(ws, name='housing')
data = data.to_pandas_dataframe()
data

def prepare_data(data):
    encoded_column=pd.get_dummies(data['ocean_proximity'], prefix='ocp')
    data=data.join(encoded_column)
    data=data.drop("ocean_proximity", axis=1)

    target="median_house_value"
    y=data[target]
    x=data.drop(target, axis=1)
    
    return x,y

def split_scale(x,y):
    x_train, x_test, y_train, y_test= train_test_split(x, y, test_size=0.2, random_state=42) # returns Dataframes and Series
    x_train = x_train.fillna(x_train.mean())
    x_test = x_test.fillna(x_test.mean())

    columns=x_train.columns.to_list()
    scaler_x = MinMaxScaler().fit(x_train[columns])
    scaled_x_train = scaler_x.transform(x_train[columns]) # numpy array
    scaled_x_test = scaler_x.transform(x_test[columns])

    

    return scaled_x_train, scaled_x_test,  y_train.values, y_test.values
x,y = prepare_data(data)
x_train, x_test, y_train, y_test  = split_scale(x,y)

In [21]:
x_train_frame=pd.DataFrame(x_train, columns=x.columns)
x_test_frame=pd.DataFrame(x_test, columns=x.columns)
x_train_frame.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocp_<1H OCEAN,ocp_INLAND,ocp_ISLAND,ocp_NEAR BAY,ocp_NEAR OCEAN
0,0.729084,0.017021,0.627451,0.079455,0.097145,0.064380,0.102286,0.190322,0.0,0.0,0.0,0.0,1.0
1,0.616534,0.129787,0.941176,0.085966,0.121974,0.036744,0.124157,0.228452,0.0,0.0,0.0,0.0,1.0
2,0.385458,0.224468,0.058824,0.048197,0.051210,0.025561,0.055090,0.252162,0.0,0.0,0.0,0.0,1.0
3,0.721116,0.014894,0.686275,0.036090,0.056797,0.039659,0.058214,0.099488,0.0,0.0,0.0,0.0,1.0
4,0.453187,0.450000,0.823529,0.060532,0.066729,0.024412,0.062325,0.210638,0.0,1.0,0.0,0.0,0.0


In [22]:
x_train_frame["median_house_value"]=y_train 
x_test_frame["median_house_value"]=y_test

x_train_frame.shape, x_test_frame.shape
x_train_frame.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocp_<1H OCEAN,ocp_INLAND,ocp_ISLAND,ocp_NEAR BAY,ocp_NEAR OCEAN,median_house_value
0,0.729084,0.017021,0.627451,0.079455,0.097145,0.064380,0.102286,0.190322,0.0,0.0,0.0,0.0,1.0,103000.0
1,0.616534,0.129787,0.941176,0.085966,0.121974,0.036744,0.124157,0.228452,0.0,0.0,0.0,0.0,1.0,382100.0
2,0.385458,0.224468,0.058824,0.048197,0.051210,0.025561,0.055090,0.252162,0.0,0.0,0.0,0.0,1.0,172600.0
3,0.721116,0.014894,0.686275,0.036090,0.056797,0.039659,0.058214,0.099488,0.0,0.0,0.0,0.0,1.0,93400.0
4,0.453187,0.450000,0.823529,0.060532,0.066729,0.024412,0.062325,0.210638,0.0,1.0,0.0,0.0,0.0,96500.0


In [23]:
x_train_frame.to_csv('train.csv')
x_test_frame.to_csv('test.csv')

In [25]:
train=pd.read_csv("train.csv",index_col=0)
train.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocp_<1H OCEAN,ocp_INLAND,ocp_ISLAND,ocp_NEAR BAY,ocp_NEAR OCEAN,median_house_value
0,0.729084,0.017021,0.627451,0.079455,0.097145,0.064380,0.102286,0.190322,0.0,0.0,0.0,0.0,1.0,103000.0
1,0.616534,0.129787,0.941176,0.085966,0.121974,0.036744,0.124157,0.228452,0.0,0.0,0.0,0.0,1.0,382100.0
2,0.385458,0.224468,0.058824,0.048197,0.051210,0.025561,0.055090,0.252162,0.0,0.0,0.0,0.0,1.0,172600.0
3,0.721116,0.014894,0.686275,0.036090,0.056797,0.039659,0.058214,0.099488,0.0,0.0,0.0,0.0,1.0,93400.0
4,0.453187,0.450000,0.823529,0.060532,0.066729,0.024412,0.062325,0.210638,0.0,1.0,0.0,0.0,0.0,96500.0


In [26]:
datastore=ws.get_default_datastore()
datastore.upload_files(['train.csv'])

Uploading an estimated of 1 files
Uploading train.csv
Uploaded train.csv, 1 files out of an estimated total of 1
Uploaded 1 files


$AZUREML_DATAREFERENCE_workspaceblobstore

In [27]:
datastore.upload_files(['test.csv'])

Uploading an estimated of 1 files
Uploading test.csv
Uploaded test.csv, 1 files out of an estimated total of 1
Uploaded 1 files


$AZUREML_DATAREFERENCE_workspaceblobstore

In [28]:
os.listdir(os.curdir)

['.amlignore',
 '.amlignore.amltmp',
 '.ipynb_aml_checkpoints',
 'hyperparameter_tuning (1).ipynb',
 'hyperparameter_tuning (1).ipynb.amltmp',
 'keras_env.yml',
 'keras_env.yml.amltmp',
 'keras_train.py',
 'keras_train.py.amltmp',
 'myenv.yml',
 'myenv.yml.amltmp',
 'scoring.py',
 'test.csv',
 'test.pkl',
 'train.csv',
 'train.pkl']

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [29]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
policy =  BanditPolicy(evaluation_interval=2, slack_factor=0.1, slack_amount=None, delay_evaluation=0)

#TODO: Create the different params that you will be using during training
ps = RandomParameterSampling(
    {
        '--batch-size': choice(15,25, 50),
        '--number-epochs': choice(5,10,15),
        '--first-layer-neurons': choice(range(2, 6, 2)),
        '--second-layer-neurons': choice(range(2,6,2))
    }
)

#TODO: Create your estimator and hyperdrive config
src = ScriptRunConfig(source_directory=project_folder,
                      script='keras_train.py',
                      compute_target=compute_target,
                      environment=keras_env
                      )

hyperdrive_config = HyperDriveConfig(
    hyperparameter_sampling = ps, 
    primary_metric_name ='MAE', 
    primary_metric_goal = PrimaryMetricGoal.MINIMIZE, 
    max_total_runs = 8, 
    max_concurrent_runs=4, 
    policy=policy, 
    run_config=src
)

## Submit run and Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [30]:
#TODO: Submit your experiment
hyperdrive_run = experiment.submit(hyperdrive_config, show_output=True)
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [31]:
hyperdrive_run.wait_for_completion(show_output=True)

RunId: HD_b2fcabd5-082a-4536-a656-ec6cf052a0de
Web View: https://ml.azure.com/runs/HD_b2fcabd5-082a-4536-a656-ec6cf052a0de?wsid=/subscriptions/48a74bb7-9950-4cc1-9caa-5d50f995cc55/resourcegroups/aml-quickstarts-247912/workspaces/quick-starts-ws-247912&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2024-01-03T13:55:03.685316][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2024-01-03T13:55:04.0384015Z][SCHEDULER][INFO]Scheduling job, id='HD_b2fcabd5-082a-4536-a656-ec6cf052a0de_0' 
[2024-01-03T13:55:04.2558237Z][SCHEDULER][INFO]Scheduling job, id='HD_b2fcabd5-082a-4536-a656-ec6cf052a0de_1' 
[2024-01-03T13:55:04.3358830Z][SCHEDULER][INFO]Scheduling job, id='HD_b2fcabd5-082a-4536-a656-ec6cf052a0de_2' 
[2024-01-03T13:55:04.4858544Z][SCHEDULER][INFO]Scheduling job, id='HD_b2fcabd5-082a-4536-a656-ec6cf052a0de_3' 
[2024-01-03T13:55:04.416788][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution t

{'runId': 'HD_b2fcabd5-082a-4536-a656-ec6cf052a0de',
 'target': 'notebook247912',
 'status': 'Completed',
 'startTimeUtc': '2024-01-03T13:55:02.933069Z',
 'endTimeUtc': '2024-01-03T13:57:35.047764Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"MAE","goal":"minimize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': 'd6dd607e-de4c-4d56-b24e-03d9200c7ce9',
  'user_agent': 'python/3.8.5 (Linux-5.15.0-1040-azure-x86_64-with-glibc2.10) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.51.0',
  'space_size': '36',
  'best_child_run_id': 'HD_b2fcabd5-082a-4536-a656-ec6cf052a0de_2',
  'score': '95059.7109375',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_b2fcabd5-082a-4536-a656-ec6cf052a0de_2'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'amlClientType': '

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [32]:
best_run=hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
best_run_metrics

{'Epochs': 10,
 'Batch Size': 15,
 'First hidden layer': 4,
 'Second hidden layer': 4,
 'Loss': 17692137654.573643,
 'MAE': 95059.7109375}

In [33]:
print('Bets Run ID', best_run.id)
print('\n MAE', best_run_metrics['MAE'])

Bets Run ID HD_b2fcabd5-082a-4536-a656-ec6cf052a0de_2

 MAE 95059.7109375


In [34]:
print(best_run.get_details()['runDefinition']['arguments'])

['--batch-size', '15', '--first-layer-neurons', '4', '--number-epochs', '10', '--second-layer-neurons', '4']


In [35]:
os.listdir(os.curdir)

['.amlignore',
 '.amlignore.amltmp',
 '.ipynb_aml_checkpoints',
 'hyperparameter_tuning (1).ipynb',
 'hyperparameter_tuning (1).ipynb.amltmp',
 'keras_env.yml',
 'keras_env.yml.amltmp',
 'keras_train.py',
 'keras_train.py.amltmp',
 'myenv.yml',
 'myenv.yml.amltmp',
 'scoring.py',
 'test.csv',
 'test.pkl',
 'train.csv',
 'train.pkl']

In [36]:
#TODO: Save the best model
# in keras_housing_train.py the model was saved in ./outputs_keras/model/

# Register model
model = best_run.register_model(model_name='keras-housing', model_path='./outputs/model')# Print model summary
model.summary()

In [37]:
from azureml.core.conda_dependencies import CondaDependencies

cd = CondaDependencies.create()
cd.add_tensorflow_conda_package()
cd.add_conda_package('h5py<=2.10.0')
cd.add_conda_package('keras<=2.3.1')
cd.add_pip_package("azureml-defaults")
cd.save_to_file(base_directory='./', conda_file_path='myenv.yml')

print(cd.serialize_to_string())

# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.8 and later.
- python=3.8.13

- pip:
  - azureml-defaults
- tensorflow=2.2.0
- h5py<=2.10.0
- keras<=2.3.1
channels:
- anaconda
- conda-forge



In [38]:
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig
from azureml.core.model import Model
from azureml.core.environment import Environment


myenv = Environment.from_conda_specification(name="myenv", file_path="myenv.yml")
inference_config = InferenceConfig(entry_script="scoring.py", environment=myenv)

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1,
                                               auth_enabled=True, # this flag generates API keys to secure access
                                               memory_gb=1,
                                               tags={'name': 'housing', 'framework': 'Keras'},
                                               description='Keras MLP on california housing')

service = Model.deploy(workspace=ws, 
                           name='keras-housing-svc', 
                           models=[model], 
                           inference_config=inference_config, 
                           deployment_config=aciconfig)

service.wait_for_deployment(True)
print(service.state)

azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2024-01-03 14:00:10+00:00 Creating Container Registry if not exists.
2024-01-03 14:00:10+00:00 Registering the environment.
2024-01-03 14:00:13+00:00 Building image..
2024-01-03 14:20:29+00:00 Generating deployment configuration..
2024-01-03 14:20:30+00:00 Submitting deployment to compute.
2024-01-03 14:20:33+00:00 Checking the status of deployment keras-housing-svc..
2024-01-03 14:22:47+00:00 Checking the status of inference endpoint keras-housing-svc.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


## Test the deployed model

In [45]:
import json

test_s=x_test[:5].tolist()
test_samples = json.dumps({"data": test_s})
test_samples = bytes(test_samples, encoding='utf8')

# predict using the deployed model
result = service.run(input_data=test_samples)


In [46]:
from sklearn.metrics import mean_absolute_error
mae_test = mean_absolute_error(y_test[:5], np.array(result))
print(round(mae_test, 3))

218020.2


In [47]:
result

[0, 0, 0, 0, 0]

In [42]:
key1, Key2 = service.get_keys()
print(key1)

oDpvB53CbgIisEpizw4hqSbXXobwfr5p


In [48]:
import requests

# send a random row from the test set to score
random_index = np.random.randint(0, len(x_test)-1)
input_data = "{\"data\": [" + str(list(x_test[random_index])) + "]}"

headers = {'Content-Type':'application/json', 'Authorization': 'Bearer ' + key1}

resp = requests.post(service.scoring_uri, input_data, headers=headers)

print("POST to url", service.scoring_uri)
#print("input data:", input_data)
print("label:", y_test[random_index])
print("prediction:", resp.text)

POST to url http://a2424709-c9fe-487e-9928-4fc80e30c8b5.southcentralus.azurecontainer.io/score
label: 107900.0
prediction: [0]


In [44]:

model = ws.models['keras-housing']
print("Model: {}, ID: {}".format('keras-housing', model.id))
    
webservice = ws.webservices['keras-housing-svc']
print("Webservice: {}, scoring URI: {}".format('keras-housing-svc', webservice.scoring_uri))

Model: keras-housing, ID: keras-housing:1
Webservice: keras-housing-svc, scoring URI: http://a2424709-c9fe-487e-9928-4fc80e30c8b5.southcentralus.azurecontainer.io/score


## Clean Up

In [ ]:
service.delete()

In [ ]:
compute_target.delete()